In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.classifier import ClassifierForCatsAndDogs

In [2]:
BATCH_SIZE = 16
ETA = 1e-4
EPOCHS = 20

In [3]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    loss = F.binary_cross_entropy(ps.squeeze(), y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    loss = F.binary_cross_entropy(ps.squeeze(), y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [6]:
def train():
    trainset = CatsAndDogs(mode="train")
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = CatsAndDogs(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = ClassifierForCatsAndDogs().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [7]:
train()

Epochs 1/20
Train loss: 0.69092307, train acc: 0.5077
Test loss: 0.69841562, test acc: 0.4927
Epochs 2/20
Train loss: 0.61859359, train acc: 0.5052
Test loss: 0.71909586, test acc: 0.5042
Epochs 3/20
Train loss: 0.53082603, train acc: 0.5069
Test loss: 0.75912769, test acc: 0.6046
Epochs 4/20
Train loss: 0.41191910, train acc: 0.5119
Test loss: 0.82727659, test acc: 0.6119
Epochs 5/20
Train loss: 0.30396221, train acc: 0.5296
Test loss: 0.88734561, test acc: 0.6339
Epochs 6/20
Train loss: 0.18386573, train acc: 0.5226
Test loss: 1.08427759, test acc: 0.6035
Epochs 7/20
Train loss: 0.09908524, train acc: 0.5215
Test loss: 1.15791741, test acc: 0.6122
Epochs 8/20
Train loss: 0.05271021, train acc: 0.5224
Test loss: 1.68204543, test acc: 0.5784
Epochs 9/20
Train loss: 0.02603689, train acc: 0.5174
Test loss: 1.48537129, test acc: 0.5765
Epochs 10/20
Train loss: 0.01388881, train acc: 0.5269
Test loss: 1.59793290, test acc: 0.6122
Epochs 11/20
Train loss: 0.00721093, train acc: 0.5198
Test